<a href="https://colab.research.google.com/github/arunraja-hub/Preference_Extraction/blob/master/onnx_py_tf_export.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **MNIST example**

Model is built in pytorch
Then exproted as tf model using ONNX
Finally, a mini-test is done using the exported tf model

In [1]:
%tensorflow_version 1.x
import tensorflow as tf

!pip install onnx_tf
!git clone https://github.com/onnx/onnx-tensorflow.git
!cd onnx-tensorflow
# !pip install -e ..

TensorFlow 1.x selected.
     |████████████████████████████████| 153kB 4.5MB/s 
     |████████████████████████████████| 4.8MB 8.6MB/s 
Cloning into 'onnx-tensorflow'...
remote: Enumerating objects: 1, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 4498 (delta 0), reused 0 (delta 0), pack-reused 4497
Receiving objects: 100% (4498/4498), 1.12 MiB | 3.40 MiB/s, done.
Resolving deltas: 100% (3390/3390), done.


In [2]:

import torch
import onnx
from onnx_tf.backend import prepare
# from google.colab import files




The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



/usr/local/lib/python3.6/dist-packages/onnx_tf/common/__init__.py:89: UserWarning: onnx_tf.common.get_outputs_names is deprecated. It will be removed in future release. Use TensorflowGraph.get_outputs_names instead.
  warnings.warn(message)


In [23]:


# General structure from https://github.com/pytorch/examples/blob/master/mnist/main.py
from __future__ import print_function
import argparse
import os
import math

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import CosineAnnealingLR
import torch.autograd as autograd
import numpy as np

from google.colab import files

batch_size=64
test_batch_size=1000
epochs=1
lr=0.1
momentum=0.9
wd=0.0005
no_cuda=False
seed=1
log_interval=10
save_model=True
data='../data'
sparsity=0.5

class GetSubnet(autograd.Function):
    @staticmethod
    def forward(ctx, scores, k):
        # Get the supermask by sorting the scores and using the top k%
        out = scores.clone()
        _, idx = scores.flatten().sort()
        j = int((1 - k) * scores.numel())

        # flat_out and out access the same memory.
        flat_out = out.flatten()
        flat_out[idx[:j]] = 0
        flat_out[idx[j:]] = 1

        return out

    @staticmethod
    def backward(ctx, g):
        # send the gradient g straight-through on the backward pass.
        return g, None


class SupermaskConv(nn.Conv2d):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        # initialize the scores
        self.scores = nn.Parameter(torch.Tensor(self.weight.size()))
        nn.init.kaiming_uniform_(self.scores, a=math.sqrt(5))

        # NOTE: initialize the weights like this.
        nn.init.kaiming_normal_(self.weight, mode="fan_in", nonlinearity="relu")

        # NOTE: turn the gradient on the weights off
        self.weight.requires_grad = False

    def forward(self, x):
        subnet = GetSubnet.apply(self.scores.abs(), sparsity)
        w = self.weight * subnet
        x = F.conv2d(
            x, w, self.bias, self.stride, self.padding, self.dilation, self.groups
        )
        return x

class SupermaskLinear(nn.Linear):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        # initialize the scores
        self.scores = nn.Parameter(torch.Tensor(self.weight.size()))
        nn.init.kaiming_uniform_(self.scores, a=math.sqrt(5))

        # NOTE: initialize the weights like this.
        nn.init.kaiming_normal_(self.weight, mode="fan_in", nonlinearity="relu")

        # NOTE: turn the gradient on the weights off
        self.weight.requires_grad = False

    def forward(self, x):
        subnet = GetSubnet.apply(self.scores.abs(), sparsity)
        w = self.weight * subnet
        return F.linear(x, w, self.bias)
        return x

# NOTE: not used here but we use NON-AFFINE Normalization!
# So there is no learned parameters for your nomralization layer.
class NonAffineBatchNorm(nn.BatchNorm2d):
    def __init__(self, dim):
        super(NonAffineBatchNorm, self).__init__(dim, affine=False)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = SupermaskConv(1, 32, 3, 1, bias=False)
        self.conv2 = SupermaskConv(32, 64, 3, 1, bias=False)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = SupermaskLinear(9216, 128, bias=False)
        self.fc2 = SupermaskLinear(128, 10, bias=False)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test(model, device, criterion, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target)
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


def main():

    use_cuda = not no_cuda and torch.cuda.is_available()

    torch.manual_seed(seed)

    device = torch.device("cuda" if use_cuda else "cpu")

    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST(os.path.join(data, 'mnist'), train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=batch_size, shuffle=True, **kwargs)
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST(os.path.join(data, 'mnist'), train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=test_batch_size, shuffle=True, **kwargs)


    test_dataset_array = next(iter(test_loader))[0].numpy()



    model = Net().to(device)
    # NOTE: only pass the parameters where p.requires_grad == True to the optimizer! Important!
    optimizer = optim.SGD(
        [p for p in model.parameters() if p.requires_grad],
        lr=lr,
        momentum=momentum,
        weight_decay=wd,
    )
    criterion = nn.CrossEntropyLoss().to(device)
    scheduler = CosineAnnealingLR(optimizer, T_max=epochs)
    for epoch in range(1, epochs + 1):
        train(model, device, train_loader, optimizer, criterion, epoch)
        test(model, device, criterion, test_loader)
        scheduler.step()

    if save_model:
        torch.save(model.state_dict(), "mnist_cnn.pt")

    # files.download('mnist_cnn.pt')

    print('model saved, now converting pytorch model to tf model using ONNX---------------------------------------------------------------------------------')
    model_pytorch = Net().to(device)
    model_pytorch.load_state_dict(torch.load('mnist_cnn.pt'))

    test_size = 2000

    input_size = 20


    #random eg from mnist

    dummy_input = np.random.randint(0, 100, (32, 1, 9,9))
    # test_dataset_array[3]
    # train_dataset_array = next(iter(train_loader))[0].numpy()
    print('numpy',np.shape(dummy_input))
    dummy_input=torch.from_numpy(dummy_input).float().to(device)
    # .reshape(1, -1)
    # [32, 1, 3, 3]
    dummy_output = model_pytorch(dummy_input)
    print(dummy_output)

#######################



    # Export to ONNX format
    torch.onnx.export(model_pytorch, dummy_input, 'mnist_cnn.onnx', input_names=['test_input'], output_names=['test_output'])

    model_onnx = onnx.load('mnist_cnn.onnx')

    tf_rep = prepare(model_onnx)

    # Export model as .pb file
    tf_rep.export_graph('mnist_cnn.pb')
    


if __name__ == '__main__':
    main()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.674934
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.764798
Train Epoch: 1 [1280/60000 (2%)]	Loss: 1.225548
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.928490
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.903956
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.749727
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.660732
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.695157
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.833756
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.594804
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.606686
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.709756
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.515206
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.404922
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.468419
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.610481
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.391762
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.332609
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.399348
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.408252
Train Epoch: 1 [12800/60000 (

RuntimeError: ignored

In [0]:

# %tensorflow_version 1.x
# import tensorflow as tf

def load_pb(path_to_pb):
    with tf.gfile.GFile(path_to_pb, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def, name='')
        return graph

tf_graph = load_pb('mnist_cnn.pb')
sess = tf.Session(graph=tf_graph)

# Show tensor names in graph
for op in tf_graph.get_operations():
  print(op.values())

output_tensor = tf_graph.get_tensor_by_name('test_output:0')
input_tensor = tf_graph.get_tensor_by_name('test_input:0')

output = sess.run(output_tensor, feed_dict={input_tensor: dummy_input})
print(output)

